In [1]:
# import all necessary libraries
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.svm import SVC
from skimage.feature import hog
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import joblib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_data = pd.read_csv('./Datasets/mnist_train.csv', header=None)

# Separate the header, labels, and features
train_labels = train_data.iloc[1:, 0]
train_data = train_data.iloc[1:, 1:]

# Convert train_labels to DataFrame
train_labels = pd.DataFrame(train_labels)

# check data shape
print(train_data.shape)
print(train_labels.shape)

(60000, 784)
(60000, 1)


In [3]:
# extracting HOG features from the dataset
def extract_features(features):
    list_hog_fd = []
    for feature in features:
        fd = hog(feature.reshape((28, 28)), orientations=4, pixels_per_cell=(4, 4), cells_per_block=(4, 4), visualize=False, block_norm = 'L2-Hys')
        list_hog_fd.append(fd)
    hog_features = np.array(list_hog_fd, 'float64')
    return hog_features

In [4]:
# Extract the features and labels
features = np.array(train_data, 'int16')
labels = np.array(train_labels, 'int')

hog_features = extract_features(features)

In [5]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(hog_features, labels, test_size = 0.80, random_state = 42)

# normalize the data
X_train = X_train / 255.0
X_test = X_test / 255.0

In [6]:
# checking the splits
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)

X_train shape: (12000, 1024)
y_train shape: (12000, 1)
X_test shape: (48000, 1024)
y_test shape: (48000, 1)


In [10]:
# Define the parameter grid for C
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}

# Create a GridSearchCV object
grid_search = GridSearchCV(SVC(kernel='linear'), param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train.ravel())

# Print the best parameters and the best score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation accuracy:", grid_search.best_score_)

# Get the best model
best_linear_svm = grid_search.best_estimator_

# Save the best linear SVM classifier
joblib.dump(best_linear_svm, "./Models/best_linearSVM.pkl")

Best parameters: {'C': 100}
Best cross-validation accuracy: 0.93925


['./Models/best_linearSVM.pkl']

In [11]:
#Load the classifier
model = joblib.load("./Models/best_linearSVM.pkl")

# predict
y_pred = model.predict(X_test)

# accuracy
print("accuracy:", metrics.accuracy_score(y_true=y_test, y_pred=y_pred), "\n")

accuracy: 0.9432916666666666 



In [10]:
# precision, recall and f1-score
scores=metrics.classification_report(y_test, y_pred, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
print(scores)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      4751
           1       0.11      1.00      0.20      5355
           2       0.00      0.00      0.00      4742
           3       0.00      0.00      0.00      4966
           4       0.00      0.00      0.00      4703
           5       0.00      0.00      0.00      4355
           6       0.00      0.00      0.00      4711
           7       0.00      0.00      0.00      5007
           8       0.00      0.00      0.00      4647
           9       0.00      0.00      0.00      4763

    accuracy                           0.11     48000
   macro avg       0.01      0.10      0.02     48000
weighted avg       0.01      0.11      0.02     48000



In [12]:
# Create a non-linear poly object
clf = SVC(kernel="poly")

# Perform the training
clf.fit(X_train, y_train.ravel())

# Save the classifier as pkl file
joblib.dump(clf, "./Models/nonlinear-Poly.pkl")

['./Models/nonlinear-Poly.pkl']

In [13]:
#Load the classifier
model = joblib.load("./Models/nonlinear-Poly.pkl")

# predict
y_pred = model.predict(X_test)

# accuracy
print("accuracy:", metrics.accuracy_score(y_true=y_test, y_pred=y_pred.ravel()), "\n")

accuracy: 0.9838541666666667 



In [14]:
# precision, recall and f1-score
scores=metrics.classification_report(y_test, y_pred, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
print(scores)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4751
           1       0.99      0.99      0.99      5355
           2       0.98      0.99      0.98      4742
           3       0.98      0.98      0.98      4966
           4       0.98      0.99      0.98      4703
           5       0.99      0.98      0.98      4355
           6       0.98      0.99      0.99      4711
           7       0.98      0.98      0.98      5007
           8       0.98      0.98      0.98      4647
           9       0.98      0.97      0.98      4763

    accuracy                           0.98     48000
   macro avg       0.98      0.98      0.98     48000
weighted avg       0.98      0.98      0.98     48000



In [15]:
# non-linear model
# using rbf kernel, C=1, default value of gamma
clf = SVC(kernel="rbf")

# fit
clf.fit(X_train, y_train.ravel())

# Save the classifier as pkl file
joblib.dump(clf, "./Models/nonlinear-Rbf.pkl")

['./Models/nonlinear-Rbf.pkl']

In [16]:
# Load the classifier
model = joblib.load("./Models/nonlinear-Rbf.pkl")

# predict
y_pred = model.predict(X_test)

# accuracy
print("accuracy:", metrics.accuracy_score(y_true=y_test, y_pred=y_pred.ravel()), "\n")

accuracy: 0.9823125 



In [17]:
# precision, recall and f1-score
scores=metrics.classification_report(y_test, y_pred, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
print(scores)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4751
           1       0.99      0.99      0.99      5355
           2       0.98      0.99      0.98      4742
           3       0.98      0.98      0.98      4966
           4       0.98      0.98      0.98      4703
           5       0.99      0.98      0.98      4355
           6       0.98      0.99      0.99      4711
           7       0.98      0.98      0.98      5007
           8       0.97      0.97      0.97      4647
           9       0.98      0.97      0.97      4763

    accuracy                           0.98     48000
   macro avg       0.98      0.98      0.98     48000
weighted avg       0.98      0.98      0.98     48000

